<a href="https://colab.research.google.com/github/NULabTMN/ps2-AnantShanbhag93/blob/master/LanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your task is to train *character-level* language models. 
You will train unigram, bigram, and trigram character level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [0]:
import pandas as pd
import httpimport

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [0]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.

If your machine takes a long time to perform this computation, you may save these counts to files in your github repository and load them on request. This is not necessary, however.

In [0]:
# Your code here


# print(train[0])
all_sent = []
for each_book in train:
    for each_sentence in each_book:
        all_sent.append(" ".join(each_sentence))

print(len(all_sent))

98552


In [0]:
print(all_sent[:3])

['[ Emma by Jane Austen 1816 ]', 'VOLUME I', 'CHAPTER I']


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

def get_uni_stat(listOfSentences):
    vect = CountVectorizer(ngram_range=(1,1), analyzer='char', lowercase=False)
    out = vect.fit_transform(listOfSentences).toarray()
    count_ngrams = out.sum(axis=0)
    vocab = vect.get_feature_names()
    stat_uni = dict(zip(vocab, count_ngrams))
    return stat_uni
#     total_count_uni = count_ngrams.sum(axis=0)

def get_bi_stat(listOfSentences):
    vect = CountVectorizer(ngram_range=(2,2), analyzer='char', lowercase=False)
    out = vect.fit_transform(listOfSentences).toarray()
    count_ngrams = out.sum(axis=0)
    vocab = vect.get_feature_names()
    stat_bi = dict(zip(vocab, count_ngrams))
    return stat_bi

def get_tri_stat(listOfSentences):
    vect = CountVectorizer(ngram_range=(3,3), analyzer='char', lowercase=False)
    out = vect.fit_transform(listOfSentences).toarray()
    count_ngrams = out.sum(axis=0)
    vocab = vect.get_feature_names()
    stat_tri = dict(zip(vocab, count_ngrams))
    return stat_tri
#     total_count_tri = count_ngrams.sum(axis=0)
# print(stat_uni)
# print(stat_bi)
# print(stat_tri)


In [0]:
stat_uni = get_uni_stat(all_sent)
print(len(stat_uni))
total_count_uni=(sum(stat_uni.values()))
print(total_count_uni)

95
12010505


In [0]:
stat_bi = get_bi_stat(all_sent)
print(len(stat_bi))

1718


In [0]:
stat_tri = get_tri_stat(all_sent)
print(len(stat_tri))

15900


## 1.2
Calculate the perplexity for each document in the test set using linear interpolation smoothing method. For determining λs for linear interpolation, you can divide the training data into a new training set (80%) and a held-out set (20%), then using grid search method:
Choose ~10 values of λ to test using grid search on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [0]:
import math
length = math.ceil(len(all_sent)*.8)
#print(length)
training = all_sent[:length]
held_out = all_sent[length:]
held_out_seq = " ".join(held_out)
print(len(all_sent))
print(len(training))
print(len(held_out))
# print(held_out_seq)

98552
78842
19710


In [0]:
stath_uni = get_uni_stat(training)
print(len(stath_uni))
total_count_uni_h=(sum(stath_uni.values()))
print(total_count_uni_h)

92
9594734


In [0]:
stath_bi = get_bi_stat(training)
print(len(stath_bi))

1658


In [0]:
stath_tri = get_tri_stat(training)
print(len(stath_tri))

14506


In [0]:
def get_prob_uni_h(w):
    return stath_uni.get(w, 50)/total_count_uni_h
def get_prob_bi_h(v, w):
    return stath_bi.get(v+w, 0)/stath_uni.get(v, 2)
def get_prob_tri_h(u, v, w):
    return stath_tri.get(u+v+w, 0)/stath_bi.get(u+v, 5)

In [0]:
def get_prob_uni(w):
    return stat_uni.get(w, 50)/total_count_uni
def get_prob_bi(v, w):
    return stat_bi.get(v+w, 0)/stat_uni.get(v, 2)
def get_prob_tri(u, v, w):
    return stat_tri.get(u+v+w, 0)/stat_bi.get(u+v, 5)

In [0]:
lam =  [[0.333, 0.333, 0.333],
        [0.1, 0.1, 0.8],
        [0.1, 0.8, 0.1],
        [0.8, 0.1, 0.1],
        [0.2, 0.2, 0.6],
        [0.2, 0.6, 0.2],
        [0.6, 0.2, 0.2],
        [0.25, 0.25, 0.5],
        [0.25, 0.5, 0.25],
        [0.5, 0.25, 0.25],
        [0.1, 0.3, 0.6],
        [0.15, 0.3, 0.6],
        [0.1, 0.4, 0.5]]

In [0]:
import numpy as np
def get_perplexity_for_doc(m, doc, lambda1, lambda2, lambda3, get_prob1, get_prob2, get_prob3):
    ent = 0
    for i in range(0, len(doc)-2):
        u = doc[i]
        v = doc[i+1]
        w = doc[i+2]
        prob = (l1 * get_prob3(u,v,w)) + (l2 * get_prob2(v, w)) + (l1 * get_prob1(w))
        ent = ent + np.log2(prob)
    ent =  (1/m) *  ent
    perp = 2 ** (-ent)
    return perp

    


import math
Mlen = len(held_out_seq)
list_perplexities = []
for l in lam:
    l1 = l[0]
    l2 = l[1] 
    l3 = l[2]
   # print(l1, l2, l3)
    list_perplexities.append(get_perplexity_for_doc(Mlen, held_out_seq, l1, l2, l3, get_prob_uni_h, get_prob_bi_h, get_prob_tri_h ))
print(list_perplexities)
min_index = list_perplexities.index(min(list_perplexities))
print(min_index)


    





[10.06132267740397, 33.504171418046404, 10.578923491445957, 5.971485875782114, 16.75209524441831, 10.22933310114893, 7.204841368635145, 13.401678651400935, 10.130259399278184, 8.048997844539398, 20.458654556985515, 16.883758582979464, 17.200614595895775]
3


In [0]:
lambda_selected = lam[min_index]
print(lambda_selected)

[0.8, 0.1, 0.1]


In [0]:

all_test = []
for test_document in test:
    each_doc = []
    for each_sent in test_document:
        t = " ".join(each_sent)
        each_doc.append(t)
    all_test.append(" ".join(each_doc))
#     t = " ".join(test_document)

In [0]:
import numpy as np
data_for_df = {"document":all_test, "file":test_files, "lang":"", "perp":np.nan,"correctness":False}
df = pd.DataFrame(data_for_df)
for i in range(len(df)):
    if df["file"].iloc[i].endswith('.txt'):
        df['lang'].iloc[i] = 'por'
    else:
        df['lang'].iloc[i] = 'eng'

In [0]:
for i in range(0, 220):
    each_doc = df["document"].iloc[i]
    ml = len(each_doc)
    df["perp"].iloc[i] = get_perplexity_for_doc(ml, each_doc, lambda_selected[0], lambda_selected[1],lambda_selected[2], get_prob_uni, get_prob_bi, get_prob_tri )

In [0]:
threshold = 33
for i in range(len(df)):
    if df["lang"].iloc[i] == 'por' and df["perp"].iloc[i] >= threshold:
        df['correctness'].iloc[i] = True
    elif df["lang"].iloc[i] == 'eng' and df["perp"].iloc[i] < threshold:
        df['correctness'].iloc[i] = True

In [0]:
df2 = df.copy()
df2 = df2.sort_values(by=['perp'])
print(df2[170:220])

                                              document          file lang  \
50   ( Do start fires one or two hours ahead of tim...          ce14  eng   
35   If the Cardinals heed Manager Gene Mauch of th...          ca15  eng   
132  The presidency : talking and listening Though ...          ca42  eng   
82   The most surprising thing about the Twenty-sec...          cb25  eng   
68   Too often a beginning bodybuilder has to do hi...          ce01  eng   
142  The nation the three-front war At a closed-doo...          ca37  eng   
114  To hold a herd of cattle on a new range till t...          cf35  eng   
124  Rookie Ron Nischwitz continued his pinpoint pi...          ca13  eng   
14   When Mickey Charles Mantle , the New York Yank...          ca39  eng   
59   Resentment welled up yesterday among Democrati...          ca07  eng   
69   About 70 North Providence taxpayers made appea...          ca24  eng   
178  Knowing specifically what the many feed additi...          ce27  eng   

## 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [0]:
# Your code here
def get_smooth_perplexity(doc,m):
    lambda_k = 0.1
    ent = 0
    for i in range(0, len(doc)-2):
        u = doc[i]
        v = doc[i+1]
        w = doc[i+2]
        prob = (stat_tri.get(u+v+w,0)+lambda_k)/((stat_bi.get(u+v,0)+(lambda_k*len(stat_tri))))
        ent = ent + np.log2(prob)
    ent =  (1/m) *  ent
    perp = 2 ** (-ent)
    return perp



In [0]:
# Your code here
for i in range(0, 220):
    each_doc = df["document"].iloc[i]
    ml = len(each_doc)
    df["perp"].iloc[i] = get_smooth_perplexity(each_doc, ml)

In [0]:
threshold = 40
for i in range(len(df)):
    if df["lang"].iloc[i] == 'por' and df["perp"].iloc[i] >= threshold:
        df['correctness'].iloc[i] = True
    elif df["lang"].iloc[i] == 'eng' and df["perp"].iloc[i] < threshold:
        df['correctness'].iloc[i] = True

In [0]:
df2 = df.copy()
df2 = df2.sort_values(by=['perp'])
print(df2[170:220])

                                              document          file lang  \
134  Philadelphia , Jan. 23 -- Nick Skorich , the l...          ca14  eng   
84   Mischa Elman shared last night's Lewisohn Stad...          cc11  eng   
68   Too often a beginning bodybuilder has to do hi...          ce01  eng   
33   St. Johns , Mich. , April 19 . -- A jury of se...          ca21  eng   
178  Knowing specifically what the many feed additi...          ce27  eng   
137  Austin , Texas -- Committee approval of Gov. P...          ca02  eng   
82   The most surprising thing about the Twenty-sec...          cb25  eng   
69   About 70 North Providence taxpayers made appea...          ca24  eng   
132  The presidency : talking and listening Though ...          ca42  eng   
10   City Controller Alexander Hemphill charged Tue...          ca09  eng   
100  California Democrats this weekend will take th...          cb11  eng   
126  Greer Garson , world-famous star of stage , sc...          ca29  eng   

## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

* The trigram model conditions on lot of context, i.e. given sufficient training data the counts for most of the trigrams are high and therefore tries to converge to the true value and the model has low bias. Where as unigram model completely ignores the context and tries to converge to inferior value and has high bias. A bigram model even though considers a bit of context falls between trigram and unigram models showing relatively high bias than the trigram model.

* In linear interpolation method, trigram model is given higher weightage than bigram and unigram. The disadvantage of trigram is that many times the trigrams in test is not present in the training set and we assign 0 to trigram value. Same for bigram value. In such conditions the perplexity is directly dependent on the unigram model which uses UNK feature to give some value to the unseen characters. As the unigram model has high bias , it will try to influence the linear interpolation. Because of this the margin between English and portugese articles is comparatively lesser than the margin between the same when calculated using trigram model with lambda smoothing. Even if the trigrams are present in test, the inclusion of bigram and unigram models in the linear interpolation induces a comparatively high bias.

* In Lambda smoothing we consider only trigram model. As mentioned earlier trigram model is conditioned on the context had has low bias and converges to true value. The disadvatage of not having trigrams is training set due to insufficient training data induces a relatively high variance. This issue is handled using lambda smoothing without depending on inferior models like unigram and bigram. So the trigram model with lambda smoothing has lower bias than the linear interpolation method and is capable of distringuishing between english and portugese atricles with a bigger margin of perplexity scores. 